In [1]:
import os
import sys
import numpy as np

In [2]:
prototype_dir = os.getcwd()
print(prototype_dir)

parent = os.path.dirname(prototype_dir)
parent_dir = os.path.dirname(parent)
sys.path.append(parent_dir)
print(parent_dir)

top_dir = os.path.dirname(parent_dir)
print(top_dir)

data_dir = top_dir + r'\neuroscikit_test_data\sequential_axona_sessions'
print(data_dir)

c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\prototypes\wave_form_sorter
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab
c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\sequential_axona_sessions


In [3]:
from core.data_spikes import (
    SpikeTrain,
    SpikeTrainBatch,
    Spike,
    SpikeCluster,
    SpikeClusterBatch,
)

from x_io.axona.read_tetrode_and_cut import (
    load_spike_train_from_paths,
)

c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit\prototypes\wave_form_sorter


In [4]:
files = os.listdir(data_dir)

test1_34 = []
test1_35 = []
test2_34 = []
test2_35 = []

for f in files:
    if 'Test1' in f and '34' in f:
        test1_34.append(f)
    elif 'Test1' in f and '35' in f:
        test1_35.append(f)
    elif 'Test2' in f and '34' in f:
        test2_34.append(f)
    elif 'Test2' in f and '35' in f:
        test2_35.append(f)

In [5]:
session1 = test1_34
session2 = test2_34

assert len(session1) == len(session2)

session1_tets = []
session2_tets = []

for i in range(len(session1)):
    if 'cut' in session1[i]:
        session1_cut = session1[i]
    if 'cut' in session2[i]:
        session2_cut = session2[i]
    file_session_1 = session1[i]
    file_session_2 = session2[i]
    out1 = file_session_1.split('.')[-1]
    out2 = file_session_2.split('.')[-1]
    if out1.isnumeric() and 'clu' not in file_session_1:
        session1_tets.append(session1[i])
    if out2.isnumeric() and 'clu' not in file_session_2:
        session2_tets.append(session2[i])



In [6]:
session1_cut_path = os.path.join(data_dir, session1_cut)
session1_tet_path = os.path.join(data_dir, session1_tets[0])

In [7]:
import wave


waveforms = [[] for i in range(4)]
for i in range(4):
    channel, empty_cell, cluster_label = load_spike_train_from_paths(session1_cut_path, session1_tet_path, i+1)
    cells = channel[:empty_cell][1]
    channel_waveforms = channel[:empty_cell][0][0]
    timestamps = channel[:empty_cell][0][1]
    waveforms[i] = channel_waveforms

# wave_vectors = np.array(list(zip(waveforms[0], waveforms[1], waveforms[2]))).squeeze()
# wave_vectors = wave_vectors.reshape((wave_vectors.shape[1], wave_vectors.shape[0], wave_vectors.shape[2]))

waveforms = np.array(waveforms)
waveforms = waveforms.reshape((waveforms.shape[1], waveforms.shape[0], waveforms.shape[2]))

In [8]:
# add duration
# go overwhich classes use self.timestamp
# make Neuron instead of SpikeCluster. Neuron class more extensible, for now merge SpikeTrain fxns, later will want to ahve 
# summary fxns to see easily if waveform data available or not, maybe make it optional to pass in 
# check combination of inputs, e.g. if waveforms, need timestamps
# e.g. waveforms put in dictionary that holds diff. attributes/stats that can be added to by user --> fxn
# also will ahve to add fxn to check that user can add data to the dict

# look into error raising that doesnt stop code if exists

# spike object --> neuron --> ensemble --> study
# ensemble most complex, holds covariates
# study ensures conssitency across ensembles, e.g. if one ensemble has this attribute then so do other ensembles
# extensible and flexible
# study class has functions to return e.g. sorted ensembles/neurons based on added covariate/added attribute

# inside file loading classes, make spike objects add to ensemble and in batch_load add ensembles to study

# NEw plan: Animal --> Contexts (sess  ions) --> SpikeTrain --> Spike object (event)
# Ensemble --> neuron --> spike object

# new new plan
# Study --> Animal --> Events
# Neuron is attribute of event (e.g. spike belonging to a neuron)
# polymorphism of events would be spikes (also e.g. lfp)
# Ensembles (data structure resulting from group by e.g. spatial, context, maze), can span animal ,session, study

session1_dict = {
    'sample_length': int(len(timestamps)),
    'sample_rate': timestamps[-1]/ len(timestamps),
    'spike_times': list(cells),
    'cluster_label': list(cluster_label),
    'ch1': list(waveforms[0]),
    'ch2': list(waveforms[1]),
    'ch3': list(waveforms[2]),
    'ch4': list(waveforms[3]),
}

In [10]:
cluster = SpikeCluster(session1_dict)

In [12]:
cluster.get_cluster_spike_count()

2

In [13]:
session1

['1-13_20210621-34-50x50cm-1500um-Test1.3',
 '1-13_20210621-34-50x50cm-1500um-Test1.clu.3',
 '1-13_20210621-34-50x50cm-1500um-Test1.eeg',
 '1-13_20210621-34-50x50cm-1500um-Test1.egf',
 '1-13_20210621-34-50x50cm-1500um-Test1.pos',
 '1-13_20210621-34-50x50cm-1500um-Test1.set',
 '1-13_20210621-34-50x50cm-1500um-Test1_3.cut']